In [ ]:
import numpy as np
from scipy.optimize import linprog

def simulate_cumulative_returns(mu, Sigma, years, n_scenarios, rng):
    """Generate 10-year cumulative (buy-and-hold) returns for each asset."""
    yearly = rng.multivariate_normal(mu, Sigma, size=(n_scenarios, years))
    factors = 1.0 + yearly                    # (N, years, 4)
    cum_factor = factors.prod(axis=1)         # (N, 4)  product across 10 yrs
    return cum_factor - 1.0                   # net return over the decade

def build_and_solve_lp(R0, R1, V0, S1, short_allowed=False):
    """
    Two-stage SAA LP:
      maximise (1/N) Σ  x1_sᵀ(1+R1_s)
      s.t.     Σ x0_i          = V0
               Σ x1_s_i − Σ(1+R0_s_i)x0_i = S1   ∀ scenarios s
               x ≥ 0  (unless short_allowed=True)
    """
    N, m = R0.shape                           # m = 4 assets
    n_vars = m + N*m                          # x0 + {x1^s}

    c = np.zeros(n_vars)
    for s in range(N):
        offs = m + s*m
        c[offs:offs+m] = -(1.0/N) * (1.0 + R1[s])   # maximise ⇒ negate

    rows, rhs = [], []

    row = np.zeros(n_vars);  row[:m] = 1.0
    rows.append(row);  rhs.append(V0)

    for s in range(N):
        row = np.zeros(n_vars)
        row[m + s*m : m + (s+1)*m] = 1.0            # +x1 terms
        row[:m] -= (1.0 + R0[s])                    # −(1+R0)x0 terms
        rows.append(row);  rhs.append(S1)

    A_eq, b_eq = np.vstack(rows), np.array(rhs)

    bounds = [(0, None)]*n_vars if not short_allowed else [(None, None)]*n_vars

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    if not res.success:
        raise RuntimeError(res.message)
    return res

def solve_msp(n_scenarios=50, seed=0, short_allowed=False):
    rng  = np.random.default_rng(seed)
    mu   = np.array([0.038, 0.050, 0.113, 0.001])
    Sigma = np.array([[ 0.070,-0.007, 0.015, 0.003],
                      [-0.007, 0.033,-0.012, 0.001],
                      [ 0.015,-0.012, 0.098,-0.001],
                      [ 0.003, 0.001,-0.001, 0.001]])
    V0   = 550_000.0
    years = 10
    # cumulative savings to t=10 (19 200 grows 3 % each year, deposited EOY)
    S1   = sum(19_200 * 1.03**t for t in range(years))

    R0 = simulate_cumulative_returns(mu, Sigma, years, n_scenarios, rng)
    R1 = simulate_cumulative_returns(mu, Sigma, years, n_scenarios, rng)

    res = build_and_solve_lp(R0, R1, V0, S1, short_allowed)
    m   = len(mu)
    x0  = res.x[:m]
    exp_V20 = -res.fun                       # negate back (maximization)

    return dict(x0=x0,
                exp_terminal_wealth=exp_V20,
                status=res.message)

for sd in range(10):
    out = solve_msp(n_scenarios=5000
                    , seed=sd, short_allowed=False)
    print(f"[seed {sd}]  x0 = {out['x0']}  |  E[V20] = {out['exp_terminal_wealth']:.2f}")


[seed 0]  x0 = [155338.97016928   2599.68193943 392061.34789129      0.        ]  |  E[V20] = 5556444.68
[seed 1]  x0 = [ 45364.8580849      0.        504635.1419151      0.       ]  |  E[V20] = 6427373.82
[seed 2]  x0 = [     0.      0. 550000.      0.]  |  E[V20] = 6764005.01
[seed 3]  x0 = [     0.          10695.84102532 539304.15897468      0.        ]  |  E[V20] = 6400496.40
[seed 4]  x0 = [     0.      0. 550000.      0.]  |  E[V20] = 6602320.78
[seed 5]  x0 = [     0.         254215.43682454 295784.56317546      0.        ]  |  E[V20] = 5354130.43
[seed 6]  x0 = [101170.11520784      0.         448829.88479216      0.        ]  |  E[V20] = 6240392.80
[seed 7]  x0 = [ 18508.48570387      0.         531491.51429613      0.        ]  |  E[V20] = 6588886.51
[seed 8]  x0 = [     0.      0. 550000.      0.]  |  E[V20] = 6796051.76
[seed 9]  x0 = [     0.          64685.53233853 485314.46766147      0.        ]  |  E[V20] = 6060140.00
